In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from scipy import optimize
from scipy.stats import sem

import os
import sys

import matplotlib.pyplot as plt
import matplotlib.ticker
from venn import venn, pseudovenn
import seaborn as sns
from matplotlib.collections import PathCollection

from Bio import SeqIO, SeqUtils, Seq
from Bio.SeqUtils.ProtParam import ProteinAnalysis

from addict import Dict
import json

In [3]:
sys.path.append("../")
import plotting as my_plot

sys.path.append("./mean_field_elect/")
import proteomics_functions as prot_fun
import base_classes
import morbidelli as m

# Load data

In [4]:
dfs = Dict()
dfs_mab = Dict()

file = pd.ExcelFile(f'./data/Chase SWATH 20220830.xlsx')
dfs.hccf.feed  = file.parse('HCCF-Feed')
dfs.hccf.large = file.parse('HCCF-LrgAgg')
dfs.hccf.small = file.parse('HCCF-SmAgg')
dfs.hccf.mab   = file.parse('HCCF-mAb')
dfs.hccf.spf1  = file.parse('HCCF-LMW1')
dfs.hccf.spf2  = file.parse('HCCF-LMW2') 
dfs.pavin.feed  = file.parse('PAFVIN-Feed')
dfs.pavin.large = file.parse('PAFVIN-LrgAgg')
dfs.pavin.small = file.parse('PAFVIN-SmAgg')

file = pd.ExcelFile('./data/Chase SWATH 20221128.xlsx')
dfs.pavin.mab = file.parse('PAFVIN mAb!')


for feed in ['hccf', 'pavin']:
    for frac in dfs[feed].keys():
        dfs[feed][frac].columns = ['accn', 'name', 'rep1_log2_norm_area', 'rep2_log2_norm_area', 'rep3_log2_norm_area', 'prot_mw', 'rep1_ng', 'rep2_ng', 'rep3_ng', 'ave_ng', 'cv']
        if frac == 'spf2':
            dfs[feed][frac]['ave_ppm'] = dfs[feed][frac]['ave_ng']/5.0 * 1e3 # x ng/5 ug * 1e-3 ug/ng * 1e6 ppm
        else:
            dfs[feed][frac]['ave_ppm'] = dfs[feed][frac]['ave_ng']/90.91 * 1e3 # x ng/90.91 ug * 1e-3 ug/ng * 1e6 ppm
                    
        # Get mAb as a separate df
        dfs_mab[feed][frac] = dfs[feed][frac][dfs[feed][frac]['name'].str.contains('Custom')]
        dfs_mab[feed][frac].reset_index(inplace=True, drop=True)
        
        # Select only CHO HCPs
        dfs[feed][frac] = dfs[feed][frac][dfs[feed][frac]['name'].str.contains('Cricetulus griseus')]
        dfs[feed][frac].reset_index(inplace=True, drop=True)

# Assemble a flat df with ave_ppm values

In [18]:
names = {}

for source in dfs.keys():
    for frac in dfs[source].keys():
        df = dfs[source][frac]
        for i, cont in df.iterrows():
            names[cont['accn']] = cont['name']
                
df_master = pd.DataFrame.from_dict(names, orient='index')
df_master.reset_index(inplace=True)
df_master.columns = ['accn', 'name']

for source in dfs.keys():
    for frac in dfs[source].keys():
        df = dfs[source][frac]
        for i, cont in df.iterrows():
            df_master.loc[df_master.accn == cont.accn, f'{source}_{frac}'] = cont.ave_ppm

In [19]:
df_master.to_csv('./generated_tables/swath_master_df_ppm_with_na.csv', index=False)

In [20]:
df_master = df_master.fillna(0)

In [21]:
df_master.to_csv('./generated_tables/swath_master_df_ppm.csv', index=False)

In [22]:
df_master.head()

,accn,name,hccf_feed,hccf_large,hccf_small,hccf_mab,hccf_spf1,hccf_spf2,pavin_feed,pavin_large,pavin_small,pavin_mab
0,NP_001230909.1,"alpha-1,3-mannosyl-glycoprotein 2-beta-N-acety...",0.308491,44.640145,2.474209,0.000000,2.526376,0.000000,0.0,0.000000,0.000000,0.0
1,NP_001230921.1,disintegrin and metalloproteinase domain-conta...,21.775462,67.131030,28.834576,1.438311,16.463027,48.661161,0.0,53.213184,8.808894,0.0
2,NP_001230924.2,MHC class I antigen Hm1-C2 precursor [Cricetul...,0.969297,19.575620,5.471594,0.000000,3.733299,0.000000,0.0,13.794253,3.384845,0.0
3,NP_001230935.1,carbonyl reductase 2 [Cricetulus griseus],2.042439,33.554075,11.558503,0.000000,21.775220,10.319524,0.0,17.617045,0.000000,0.0
4,NP_001230936.1,carbonyl reductase [NADPH] 3 [Cricetulus griseus],0.585707,37.070939,4.646791,0.000000,35.039610,24.485879,0.0,22.751988,4.818177,0.0


# Add biophysical properties

In [62]:
# # Get new html links to look up sequences

# html = ''
# cnt = 0

# for i, a in enumerate(df_master.accn):
#     if i % 200 == 0:
#         print(html[:-1], '\n'*2)
#         html = 'https://www.ncbi.nlm.nih.gov/protein/'        
#     html += a + ','
#     cnt += 1
    
# print(html[:-1], '\n'*2)

In [23]:
# My substitution rules for uncertain amino acids
my_sub_rules = {'B':'D', 'Z':'E', 'X':'A', 'J':'L'}

# Get sequence dictionary {accession:sequence_object}
sequences = {}
subbed_ids = []

for r in SeqIO.parse("./data/sequences_swath.fasta", "fasta"):
    for aa in my_sub_rules.keys(): # uncertain amino acids
        if aa in r.seq:
            r.seq = Seq.Seq(str(r.seq).replace(aa, my_sub_rules[aa]))
            subbed_ids.append(r.id)
    sequences[r.id] = r.seq

In [24]:
missing = []
for i, cont in df_master.iterrows():
    if cont.accn not in list(sequences.keys()):
        missing.append(cont.accn)
assert len(missing) == 0

In [25]:
# Get pI and mass dictionaries {accession:pI/mass}
pI_vals = {}
masses = {}

for p_id, seq in sequences.items():
    pI, is_solved = prot_fun.get_pI(seq)
    assert is_solved
    pI_vals[p_id] = pI
    masses[p_id] = SeqUtils.molecular_weight(seq, seq_type='protein')

/home/chase/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [26]:
# Add sequences, pI values, and masses to df_master
for i, cont in df_master.iterrows():
    df_master.at[i, 'sequence'] = str(sequences[cont.accn])
    df_master.at[i, 'pI'] = pI_vals[cont.accn]
    df_master.at[i, 'mass'] = masses[cont.accn]

In [27]:
# Get other biophysical property dictionaries (assuming pH 7.0)
net_charges, net_neg_charges, net_pos_charges, charge_densities, charge_densities_neg, charge_densities_pos = {}, {}, {}, {}, {}, {}

for p_id, seq in sequences.items():
    net_charge, net_neg_charge, net_pos_charge, charge_dens, charge_dens_neg, charge_dens_pos = prot_fun.get_charge(pH=7.0, seq=seq, charge_contributions=True)
    net_charges[p_id] = net_charge
    net_neg_charges[p_id] = net_neg_charge
    net_pos_charges[p_id] = net_pos_charge
    charge_densities[p_id] = charge_dens
    charge_densities_neg[p_id] = charge_dens_neg
    charge_densities_pos[p_id] = charge_dens_pos

In [28]:
# Add these biophysical properties to df_master
for i, cont in df_master.iterrows():
    df_master.at[i, 'net_charge'] = net_charges[cont.accn]
    df_master.at[i, 'net_charge_neg'] = net_neg_charges[cont.accn]
    df_master.at[i, 'net_charge_pos'] = net_pos_charges[cont.accn]
    df_master.at[i, 'charge_dens_C_m2'] = charge_densities[cont.accn]
    df_master.at[i, 'charge_dens_neg_C_m2'] = charge_densities_neg[cont.accn]
    df_master.at[i, 'charge_dens_pos_C_m2'] = charge_densities_pos[cont.accn]

In [29]:
df_master.to_csv('./generated_tables/swath_master_df_ppm_with_properties.csv', index=False)